## Arquitectura de módulos del discriminamómetro

Desde este notebook se expone la arquitectura general del proyecto "Discriminamómetro", esto es con la intención de poder correr algún proceso en específico de manera rápida y sencilla.

# ETL
En el módulo ETL se encuentra: 

-El proceso de descarga recurrente que deberá estar programado para ejecutarse cada 15 minutos mediante un cron.
 
-El proceso de obtención de tweets ya sea por usuario, o hashtag/palabras específicas

In [1]:
import Discrim.etl as etl

obj_etl = etl.etl('PRUEBA')
print(obj_etl.nbr_TweetsXCorrida)

obj_etl.descarga_recurrente_tweets()

8
Entra a descarga_recurrente_tweets
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_apariencia_20200814_214627607720.txt
str_NombreArchivoEnS3:  00_pruebas/json_apariencia_20200814_214627607720.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_discapacidad_20200814_214628281254.txt
str_NombreArchivoEnS3:  00_pruebas/json_discapacidad_20200814_214628281254.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_edad_20200814_214628901932.txt
str_NombreArchivoEnS3:  00_pruebas/json_edad_20200814_214628901932.txt
Enviando el archivo a S3...
str_ArchivoEnvio:  /home/jupyter-ronmoy007@gmail.co-0e856/discriminamometro/discriminamometro/scripts/json_genero_20200814_214629529213.txt
str_NombreArchivoEnS3:  00_pruebas/json_genero_20200814_214629529213.txt
Enviando el

# Train
El "Discriminamómetro" está constituido por 2 capas, en la primer capa se evalúa mediante un modelo si un tweet discrimina o no. En caso de que un tweet sea discriminatorio, en la segunda capa se evaluarán 7 modelos (uno por cada categoría) para determinar el grado de pertenencia a cada una de ellas. Además de ello, para lograr capturar el contexto de cada palabra, se incluye una fase de generación de embeddings que servirá para entrenar los modelos

## Embeddings

In [3]:
import Discrim.Train.embeddings as emb

obj_emb = emb.Embeddings()

obj_emb.descargar_archivo_fuente()
obj_emb.transformar_dataset()
obj_emb.generar_embeddings()
obj_emb.enviar_embeddings_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


str_ArchivoEnvio:  ./data/word2vec_Embeddings.txt
str_NombreArchivoEnS3:  00_entrenamiento/embeddings/word2vec_Embeddings.txt
Enviando el archivo a S3...
Envío completado


# Capa1

In [5]:
! gzip ./data/word2vec_Embeddings.txt

In [6]:
! python -m spacy init-model es data --vectors-loc ./data/word2vec_Embeddings.txt.gz

✔ Successfully created model
53549it [00:09, 5552.76it/s]word2vec_Embeddings.txt.gz
✔ Loaded vectors from data/word2vec_Embeddings.txt.gz
✔ Sucessfully compiled vocab
53767 entries, 53549 vectors


In [1]:
import Discrim.Train.capa1 as cp1

obj_capa1 = cp1.capa1()
obj_capa1.descargar_tweets_entrenamiento()
obj_capa1.transformar_dataset()
obj_capa1.train()
obj_capa1.crear_pickle()
obj_capa1.mandar_pickle_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 4}
grid_search.best_score_:  0.6472299043463667
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    7.3s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


grid_search.best_params_:  {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 50}
grid_search.best_score_:  0.677574089612351
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   34.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


grid_search.best_params_:  {'learning_rate': 0.1, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 16, 'n_estimators': 50}
grid_search.best_score_:  0.6936801476757846
str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


In [2]:
import numpy as np

npEmbeddings = obj_capa1.generar_embeddings(obj_capa1.test)

predicted = obj_capa1.best_model.predict(npEmbeddings)

np.mean(predicted == obj_capa1.test.label)

0.665

str_ArchivoEnvio:  modelo_capa1.p
str_NombreArchivoEnS3:  01_modelos/capa1/modelo_capa1.p
Enviando el archivo a S3...
Envío completado


# Capa2
En la segunda capa, se evaluarán 7 modelos

In [1]:
import Discrim.Train.Capa2.apariencia as apa

obj_apariencia = apa.Apariencia()
obj_apariencia.descargar_tweets_entrenamiento()
obj_apariencia.transformar_dataset()
obj_apariencia.train()
obj_apariencia.crear_pickle()
obj_apariencia.mandar_pickle_s3()

In [2]:
import Discrim.Train.Capa2.discapacidad as disc

obj_discapacidad = disc.Discapacidad()
obj_discapacidad.descargar_tweets_entrenamiento()
obj_discapacidad.transformar_dataset()
obj_discapacidad.train()
obj_discapacidad.crear_pickle()
obj_discapacidad.mandar_pickle_s3()

In [3]:
import Discrim.Train.Capa2.edad as edad

obj_edad = edad.Edad()
obj_edad.descargar_tweets_entrenamiento()
obj_edad.transformar_dataset()
obj_edad.train()
obj_edad.crear_pickle()
obj_edad.mandar_pickle_s3()

In [5]:
import Discrim.Train.Capa2.genero as gen

obj_genero = gen.Genero()
obj_genero.descargar_tweets_entrenamiento()
obj_genero.transformar_dataset()
obj_genero.train()
obj_genero.crear_pickle()
obj_genero.mandar_pickle_s3()

[nltk_data] Downloading package punkt to /home/jupyter-
[nltk_data]     ronmoy007@gmail.co-0e856/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    1.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 16}
grid_search.best_score_:  0.5657705206378987
Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    5.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


grid_search.best_params_:  {'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 50}
grid_search.best_score_:  0.5509621638524078
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   18.2s finished
/home/jupyter-ronmoy007@gmail.co-0e856/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


grid_search.best_params_:  {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 16, 'n_estimators': 50}
grid_search.best_score_:  0.5678391963727331
str_ArchivoEnvio:  modelo_capa2_genero.p
str_NombreArchivoEnS3:  01_modelos/capa2/genero/modelo_capa2_genero.p
Enviando el archivo a S3...
Envío completado


In [4]:
import Discrim.Train.Capa2.ideologia as ideo

obj_ideologia = ideo.Ideologia()
obj_ideologia.descargar_tweets_entrenamiento()
obj_ideologia.transformar_dataset()
obj_ideologia.train()
obj_ideologia.crear_pickle()
obj_ideologia.mandar_pickle_s3()

In [5]:
import Discrim.Train.Capa2.orientacion as orien

obj_orientacion = orien.Orientacion()
obj_orientacion.descargar_tweets_entrenamiento()
obj_orientacion.transformar_dataset()
obj_orientacion.train()
obj_orientacion.crear_pickle()
obj_orientacion.mandar_pickle_s3()

In [6]:
import Discrim.Train.Capa2.religion as relig

obj_religion = relig.Religion()
obj_religion.descargar_tweets_entrenamiento()
obj_religion.transformar_dataset()
obj_religion.train()
obj_religion.crear_pickle()
obj_religion.mandar_pickle_s3()

In [6]:
obj_genero.best_model

GradientBoostingClassifier(max_features='sqrt', min_samples_leaf=3,
                           min_samples_split=16, n_estimators=50)

In [3]:
obj_genero.pd_fuente

,Unnamed: 0,full_text,categoria_1-ap,categoria_2-dis,categoria_3-ed,label,categoria_5-id,categoria_6-or,categoria_7-rel
0,0,[] proximamente rd jodido el4tobate,1,0,0,0,0,0,0
1,1,jajajaja acomplejado guerito rockero ja facil ...,1,0,0,0,0,0,0
2,2,momentos cuenta contados dedos personas sincer...,1,0,0,0,0,0,0
3,3,guerito,1,0,0,0,0,0,0
4,4,restriccion preparado casos covid-19 suben 15 ...,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
495,497,parece super nefasto gente llame echando ganas...,1,0,0,0,0,0,0
496,498,super caga tipo creen super super perrisimas a...,1,0,0,0,0,0,0
497,499,mejor amlrover incluye chofer come duele mante...,1,0,0,0,0,0,0
498,500,ahhhhh quedes 1 culpa cajeros 49 cambio sufici...,1,0,0,0,0,0,0


# Predict

In [4]:
import Discrim.Predict as pr

# Dashboard

In [6]:
import Discrim.Dashboard as dash

# API

In [7]:
import Discrim.API as api